<a href="https://colab.research.google.com/github/vedanth-i/483-final-project/blob/main/483_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
#
# geometric_libs = '/content/notebooks'
# os.symlink("/content/drive/MyDrive/Colab Notebooks", geometric_libs)
# sys.path.insert(0,geometric_libs)

In [44]:
import os
import torch
print("Using torch", torch.__version__)

Using torch 2.1.0+cu118


In [45]:
!pip install --target=$geometric_libs torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
#can skip this step next time since already installed on drive
#!pip install -r "/content/drive/MyDrive/Colab Notebooks/requirements.txt"

In [46]:
!pip install --target=$geometric_libs ogb
# !pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
# !pip install ogb  # for datasets

In [47]:
import torch_scatter, torch_sparse, torch_cluster, torch_spline_conv, torch_geometric, ogb

In [48]:
# import torch_geometric.data into environment
from torch_geometric.data import Data
from torch_geometric import nn
import torch_geometric.transforms as T
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

In [49]:
from torch_geometric.datasets import Amazon

In [50]:
dataset_computers = Amazon('/tmp/amazoncomputers', 'Computers')

In [51]:
dataset_photos = Amazon('/tmp/amazonphotos', 'Photo')

In [52]:
dataset_computers.num_classes, dataset_photos.num_classes

(10, 8)

In [53]:
class MLP1(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()

        self.layer_1 = nn.Linear(input_dim, hidden_dim)
        self.layer_2 = nn.Linear(hidden_dim, output_dim)
        self.act = nn.ReLU()


    def forward(self, x):

        x = x.view(1, -1)
        out = 0

        out = self.layer_1(x) #linear 1
        out = self.act(out) #relu
        out = self.layer_2(out) #linear 2

        return out

In [54]:
#CHANGE HERE these values to get results

hidden_channels = 200 #can change number of hidden channels for variability
l_rate = 0.01 #learning rate
split = 11000 #train/test split.
epochs = 50

In [55]:
#Computers dataset

num_features_computers = dataset_computers.num_features
num_classes_computers = dataset_computers.num_classes # please write down the number of classes



In [56]:
#Photos dataset


num_features_photos = dataset_photos.num_features
num_classes_photos = dataset_photos.num_classes # please write down the number of classes



In [57]:
#define model here

#MLP Model

model_1_computers = MLP1(num_features_computers, hidden_channels, num_classes_computers)
model_1_photos = MLP1(num_features_photos, hidden_channels, num_classes_photos)


In [58]:
optimizer_1_computers = optim.Adam(model_1_computers.parameters(), lr=l_rate)
optimizer_1_photos = optim.Adam(model_1_photos.parameters(), lr=l_rate) #play around with the learning rate for differentiating

loss_fn = nn.CrossEntropyLoss()

In [59]:
def train(model, data, optimizer, loss_fn):

    loss = 0
    split = len(data.y)

    model.train()

    for i in range(0, split): #train/test split is 0.8/0.2. For computers, 0.8*13,752 = 11000. For photos 0.8*7650 = 6048

      optimizer.zero_grad()

      y_pred = model(data.x[i])


      y_actual = torch.zeros(1,10)
      y_actual[0][data.y[i]] = 1

      torch.squeeze(y_pred, dim=0)
      #print(y_pred.shape, data.y.shape, data.y, y_pred)

      l = loss_fn(y_pred, y_actual)  # calculate the loss

      l.backward()  # calculate the gradients of each parameter

      optimizer.step()  # update the parameters by taking an optimizer step

      loss += l.item()


    #########################################

    return loss

In [60]:
dataset_computers.data, dataset_computers.num_features, dataset_computers.num_classes

(Data(x=[13752, 767], edge_index=[2, 491722], y=[13752]), 767, 10)

In [61]:
dataset_photos.data, dataset_photos.num_features, dataset_photos.num_classes

(Data(x=[7650, 745], edge_index=[2, 238162], y=[7650]), 745, 8)

In [62]:
def calculate_accuracy(pred, label):

  acc = 0.0

  new = torch.softmax(pred, dim=1).argmax(dim=1)
  # print(new, new.shape)
  # print(label.shape)
  final = torch.eq(new, label).sum().item()
  acc = final/len(pred)


  return acc

In [63]:
def evaluate(model, data, loss_fn):

    loss = 0
    epoch_acc = 0
    split = 11000

    model.eval()
    for i in range(split, len(data.y)): #0.2 split for testing. Remainder of examples used. For computers, ~2750 examples. For photos 1512

      y_pred = model(data.x[i])

      y_actual = torch.zeros(1,10)
      y_actual[0][data.y[i]] = 1

      #print(y_pred.shape, data.y.shape, data.y, y_pred)

      l = loss_fn(y_pred, y_actual)
      loss += l.item()
      epoch_acc += calculate_accuracy(y_pred, y_actual)


    return epoch_acc / len(data.y)

In [ ]:
best_val_acc = final_test_acc = 0
for epoch in range(1, epochs + 1):
    loss = train(model_1_computers, dataset_computers.data, optimizer_1_computers, loss_fn)
    print(loss)


21785.21992010533
20290.932521068968
19739.77181237738
19008.79474176968
18661.989557041805
17030.822358881418


In [ ]:
#Save model's parameters like such. Can change parameters depending on what model you want to save
torch.save(model_1_photos.state_dict(), 'model_photos_200_0.001.pt')  # saving model's parameters

In [ ]:
test_acc = evaluate(model_1_computers, dataset_computers.data, loss_fn)
print(test_acc)

In [ ]:
from torch_geometric.nn import GCNConv

class GCN1(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        self.conv1 = GCNConv(in_channels, hidden_channels)

        self.conv3 = GCNConv(hidden_channels, hidden_channels)

        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.act = torch.nn.ReLU()

    def forward(self, node_feature, edge_index):

        output = self.conv1(node_feature, edge_index)
        output = self.act(output)

        #Can add/remove GCN layers

        # output = self.conv3(output, edge_index)
        # output = self.act(output)
        # output = self.conv3(output, edge_index)
        # output = self.act(output)

        # output = self.conv3(output, edge_index)
        # output = self.act(output)
        # output = self.conv3(output, edge_index)
        # output = self.act(output)

        # output = self.conv3(output, edge_index)
        # output = self.act(output)
        # output = self.conv3(output, edge_index)
        # output = self.act(output)


        output = self.conv2(output, edge_index)

        return output

In [ ]:
#CHANGE HERE these values to get results

hidden_channels = 200 #can change number of hidden channels for variability
l_rate = 0.01 #learning rate
split = 11000 #train/test split
epochs = 50

In [ ]:

#Computers and Photos

num_features_computers = dataset_computers.num_features
num_classes_computers = dataset_computers.num_classes # please write down the number of classes

num_features_photos = dataset_photos.num_features
num_classes_photos = dataset_photos.num_classes # please write down the number of classes



In [ ]:
#GCN Models
gcn_computers = GCN1(num_features_computers, hidden_channels, num_classes_computers)
gcn_photos = GCN1(num_features_photos, hidden_channels, num_classes_photos)

In [ ]:
#GCN Optimizers
gcn_opt_computers = optim.Adam(gcn_computers.parameters(), lr=l_rate)
gcn_opt_photos = optim.Adam(gcn_photos.parameters(), lr=l_rate) #play around with the learning rate for differentiating

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def trainGCN(model, data, optimizer, loss_fn):

    loss = 0

    model.train()

    optimizer.zero_grad()

    y_pred = model(data.x, data.edge_index)

    l = loss_fn(y_pred[:split], data.y.view(-1)[:split])  # calculate the loss

    l.backward()  # calculate the gradients of each parameter

    optimizer.step()  # update the parameters by taking an optimizer step

    loss += l.item()


    #########################################

    return loss

In [ ]:
def testGCN(model, data):


  acc = 0
  model.eval()
  pred = model(data.x, data.edge_index)
  print(pred.shape)

  predicted_classes = torch.argmax(pred[split:], dim=1)
  acc = torch.mean((predicted_classes == data.y[split:]).float()).item()

  #########################################

  return acc


In [ ]:
for epoch in range(1, epochs + 1):
    loss = trainGCN(gcn_computers, dataset_computers.data, gcn_opt_computers, loss_fn)
    print(loss)

In [ ]:
torch.save(gcn_computers.state_dict(), 'gcn_computers_layers.pt')  # saving model's parameters

In [ ]:
test_acc = testGCN(gcn_computers, dataset_computers.data)
print(test_acc)